In [ ]:
import numpy as np
import os
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
nturn = 2048
npt = 64
std = 0.2
Espread = 2.0e-3
OnEnergy = False

# Read MLI input for settings

In [ ]:
elems,latticeList,labor = mli.readInputfile('mli.in.t3_iota_8_4_t0p4')
# elems[2].driftexact=1
elems,latticeList=mli.sext2thin(elems,latticeList,brho=0.50204778184582999)

In [ ]:
elemList = []
for item in elems:
    if not item.elem in ['tasm','aim','vary','anaprint','monitor']:
        elemList.append(item)

In [ ]:
energy=elems[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [ ]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx


f3 = k/(2*bet0)
f4 = k/(2*bet0*gam0)**2
print(bet0,k,f3,f4)

In [ ]:
def MLI2norm(data_in,sign=1):
    data=data_in.copy()
    data[:,5] = np.sqrt(1.0-2.0*data[:,5]/bet0+data[:,5]**2)-1.0
    data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    return data
    
def norm2MLI(data_in,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
    data[:,5] = -np.sqrt((1.0+data[:,5])**2+1.0/(bet0*gam0)**2)+1.0/bet0
    return data

In [ ]:
from scipy.stats import truncnorm

def get_truncated_normal(mean=0, sd=1, low=0, upp=10,n=1):
    f = truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
    return f.rvs(n)

In [ ]:
# pData0 = np.load('pDataNorm.npy')
# pDataIn = norm2MLI(pData0,sign=1)
# np.savetxt('rays.in',pDataIn)

In [ ]:
x=get_truncated_normal(sd=std,low=-3.0*std,upp=3.0*std,n=npt*5)
pData=np.zeros([npt,6])
pData[:,[0,1,2,3,5]]=x.reshape([npt,5])
pData[:,5] = pData[:,5]/std*Espread
# np.save('pDataNorm.std=0.1',pData)
# pData = np.load('pDataNorm.std=0.1.npy')
if OnEnergy:
    pData[:,5] = 0.0

pDataIn = norm2MLI(pData,sign=1)
np.savetxt('rays.in',pDataIn)

# Cmds

In [ ]:
raysin  =mli.getElem.raytrace(file1='rays.in',type='readonly')
dump    =mli.getElem.particledump(file='rays.out',precision=15)
track   =mli.getElem.autotrack(type='symplectic',order=5)
fin     =mli.getElem.fin

elemList = elemList + [raysin, dump, track, fin]

In [ ]:
labor = mli.buildLabor(['iotaline','mapout'])
mli.writeInputfile(elemList,latticeList,labor)
mli.run()

# Lattice

In [ ]:
nlinsert=mli.getElem.nlinsert()

iotaline = latticeList[0]
arc = copy(iotaline)
arc.name = 'arc'
iotaline.list.insert(0,nlinsert.name)
iotaline.list.append(dump.name)

elemList = elemList + [nlinsert]

In [ ]:
indexThin = []
for i,item in enumerate(elemList):
    if 'thlm' == item.elem:
        indexThin.append(i)
nThin = len(indexThin)
print(nThin)

# Labor

In [ ]:
labor = mli.buildLabor(['iotaline','mapout','clear',raysin.name,track.name,str(nturn)+'*'+iotaline.name,'fin'])
maplabor = mli.buildLabor(['arc','mapout'])

#  Run

In [ ]:
def getTBT(npt,nturn,fname='rays.out'):
    TBT = np.loadtxt(fname)
    TBT = TBT[:npt*nturn,:6]
    out = np.zeros([npt,nturn,6])
    for i in range(nturn):
        out[:,i,:] = TBT[i*npt:(i+1)*npt,:]
        out[:,i,:] = MLI2norm(out[:,i,:])
    return out

In [ ]:
def updateStrength(arg):
    for i,j in enumerate(indexThin):
        elemList[j]['k2l']=arg[i]

In [ ]:
updateStrength([0.0]*nThin)
mli.writeInputfile(elemList,latticeList,labor)
mli.run(8)
!mv mli.log mli.0.log
TBT0 = getTBT(npt,nturn)
print(TBT0.shape)

In [ ]:
import pickle
# with open('result.thin.sext.oct','rb') as fp:
# with open('result.thin.DA.sext','rb') as fp:
with open('result.Inv.thin.1sig.sext','rb') as fp:
    result=pickle.load(fp)

In [ ]:
# updateStrength(result.x)
arg = result.population[0,:]
arg = arg*200-100
updateStrength(arg)
mli.writeInputfile(elemList,latticeList,labor)
mli.run(8)
!mv mli.log mli.1.log
TBT1 = getTBT(npt,nturn)
print(TBT1.shape)

In [ ]:
mli.writeInputfile(elemList,[arc],maplabor)
mli.run()
M,G=mli.readTransferMap()

In [ ]:
readmap = mli.getElem.tmi(name='readmap')
readmap.map2file(M,G)
savemap = mli.getElem.stm(name='savemap')
arcM    = mli.getElem.gtm(name='arcM')
elemList = elemList + [readmap,savemap,arcM]
iotaline.list=[iotaline.list[0],'arcM',iotaline.list[-1]]
labor.list = ['readmap','savemap','arcM','clear'] + labor.list[-4:]
mli.writeInputfile(elemList,latticeList,labor)
mli.run(8)
!mv mli.log mli.2.log
TBT2 = getTBT(npt,nturn)
print(TBT2.shape)

# Check invariants

In [ ]:
def getInv(xn,pxn,yn,pyn,delta,tau=NL_t):
    z = xn + 1j*yn
    U = np.real(z/np.sqrt(1-z**2)*np.arcsin(z))
    W = np.real(2*xn/np.sqrt(1-z**2)*np.arcsin(z))
    Hn = 0.5*(xn**2+pxn**2+yn**2+pyn**2)   +tau*U/(1.0+delta)
    In = (xn*pyn -yn*pxn)**2 +xn**2+pxn**2 +tau*W/(1.0+delta)
    return Hn,In

def getInvTBT(TBT):
    npt,nturn,dummy = TBT.shape
    InvTBT = np.zeros([npt,nturn,2])
    for iturn in range(nturn):
        data = TBT[:,iturn,:]
        for ipt in range(npt):
            xn    = data[ipt,0]
            pxn   = data[ipt,1]
            yn    = data[ipt,2]
            pyn   = data[ipt,3]
            delta = data[ipt,5]
            InvTBT[ipt,iturn,:] = getInv(xn,pxn,yn,pyn,delta)
    return InvTBT

In [ ]:
Inv0 = getInvTBT(TBT0)
Inv1 = getInvTBT(TBT1)
Inv2 = getInvTBT(TBT2)

### Absolute

In [ ]:
print('--max invarinats--')
print(np.max(Inv0[:,:,0]),np.max(Inv0[:,:,1]))
print(np.max(Inv1[:,:,0]),np.max(Inv1[:,:,1]))
print(np.max(Inv2[:,:,0]),np.max(Inv2[:,:,1]))

In [ ]:
amp0 = np.zeros([npt,2])
amp1 = np.zeros([npt,2])
amp2 = np.zeros([npt,2])
for i in range(npt):
    amp0[i,0] = np.std(Inv0[i,:,0])
    amp0[i,1] = np.std(Inv0[i,:,1])
    amp1[i,0] = np.std(Inv1[i,:,0])
    amp1[i,1] = np.std(Inv1[i,:,1])
    amp2[i,0] = np.std(Inv2[i,:,0])
    amp2[i,1] = np.std(Inv2[i,:,1])
print('--mean of amplitudes--')
print(amp0[:,0].mean(),amp0[:,1].mean())
print(amp1[:,0].mean(),amp1[:,1].mean())
print(amp2[:,0].mean(),amp2[:,1].mean())

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
for i in range(npt):
    plt.plot(Inv0[i,:,0],marker='.',ls='',alpha=0.7)
plt.subplot(1,2,2)
for i in range(npt):
    plt.plot(Inv0[i,:,1],marker='.',ls='',alpha=0.7)
plt.tight_layout()

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
for i in range(npt):
    plt.plot(Inv1[i,:,0],marker='.',ls='',alpha=0.7)
plt.subplot(1,2,2)
for i in range(npt):
    plt.plot(Inv1[i,:,1],marker='.',ls='',alpha=0.7)
plt.tight_layout()

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
for i in range(npt):
    plt.plot(Inv2[i,:,0],marker='.',ls='',alpha=0.7)
plt.subplot(1,2,2)
for i in range(npt):
    plt.plot(Inv2[i,:,1],marker='.',ls='',alpha=0.7)
plt.tight_layout()

### Relative

In [ ]:
for i in range(npt):
    Inv0[i,:,0] = Inv0[i,:,0]/Inv0[i,0,0].mean()-1.0
    Inv0[i,:,1] = Inv0[i,:,1]/Inv0[i,0,1].mean()-1.0
    Inv1[i,:,0] = Inv1[i,:,0]/Inv1[i,0,0].mean()-1.0
    Inv1[i,:,1] = Inv1[i,:,1]/Inv1[i,0,1].mean()-1.0
    Inv2[i,:,0] = Inv2[i,:,0]/Inv2[i,0,0].mean()-1.0
    Inv2[i,:,1] = Inv2[i,:,1]/Inv2[i,0,1].mean()-1.0

In [ ]:
print('--max invarinats--')
print(np.max(Inv0[:,:,0]),np.max(Inv0[:,:,1]))
print(np.max(Inv1[:,:,0]),np.max(Inv1[:,:,1]))
print(np.max(Inv2[:,:,0]),np.max(Inv2[:,:,1]))

In [ ]:
amp0 = np.zeros([npt,2])
amp1 = np.zeros([npt,2])
amp2 = np.zeros([npt,2])
for i in range(npt):
    amp0[i,0] = np.std(Inv0[i,:,0])
    amp0[i,1] = np.std(Inv0[i,:,1])
    amp1[i,0] = np.std(Inv1[i,:,0])
    amp1[i,1] = np.std(Inv1[i,:,1])
    amp2[i,0] = np.std(Inv2[i,:,0])
    amp2[i,1] = np.std(Inv2[i,:,1])
print('--mean of amplitudes--')
print(amp0[:,0].mean(),amp0[:,1].mean())
print(amp1[:,0].mean(),amp1[:,1].mean())
print(amp2[:,0].mean(),amp2[:,1].mean())

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
for i in range(npt):
    plt.plot(Inv0[i,:,0],marker='.',ls='',alpha=0.7)
plt.subplot(1,2,2)
for i in range(npt):
    plt.plot(Inv0[i,:,1],marker='.',ls='',alpha=0.7)
plt.tight_layout()

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
for i in range(npt):
    plt.plot(Inv1[i,:,0],marker='.',ls='',alpha=0.7)
plt.subplot(1,2,2)
for i in range(npt):
    plt.plot(Inv1[i,:,1],marker='.',ls='',alpha=0.7)
plt.tight_layout()

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
for i in range(npt):
    plt.plot(Inv2[i,:,0],marker='.',ls='',alpha=0.7)
plt.subplot(1,2,2)
for i in range(npt):
    plt.plot(Inv2[i,:,1],marker='.',ls='',alpha=0.7)
plt.tight_layout()